In [32]:
import numpy as np
import random as rd
from math import ceil
from functools import reduce
from queue import SimpleQueue, PriorityQueue
from collections import namedtuple
import time


In [56]:

PROBLEM_SIZE = 30
NUM_SETS = 50
SETS = tuple([np.array([rd.random() < 0.15 for _ in range(PROBLEM_SIZE)]) for _ in range(NUM_SETS)])

initial_state = (set(), set(range(NUM_SETS)))

In [4]:
def covered(state):
    return reduce(np.logical_or, [SETS[i] for i in state[0]], np.array([False for _ in range(PROBLEM_SIZE)]))

def goal_check(state):
    return np.all(covered(state))

def distance(state):
    return PROBLEM_SIZE - np.sum(covered(state))

def h1(state):
    largest_set_size = max(sum(s) for s in SETS)
    missing_size = PROBLEM_SIZE - sum(covered(state))
    optimistic_estimate = ceil(missing_size / largest_set_size)
    return optimistic_estimate


def h2(state):
    already_covered = covered(state)
    if np.all(already_covered):
        return 0
    largest_set_size = max(sum(np.logical_and(s, np.logical_not(already_covered))) for s in SETS)
    missing_size = PROBLEM_SIZE - sum(already_covered)
    optimistic_estimate = ceil(missing_size / largest_set_size)
    return optimistic_estimate


def h3(state):
    already_covered = covered(state)
    if np.all(already_covered):
        return 0
    missing_size = PROBLEM_SIZE - sum(already_covered)
    candidates = sorted((sum(np.logical_and(s, np.logical_not(already_covered))) for s in SETS), reverse=True)
    taken = 1
    while sum(candidates[:taken]) < missing_size:
        taken += 1
    return taken

def h4(state): #not worth compared to h3
    filtered_sets = [SETS[i] for i in state[1]]
    covered_elements = covered(state)
    missing_elements = PROBLEM_SIZE - sum(covered_elements)
    steps = 0

    while missing_elements > 0 and filtered_sets:
      filtered_sets = sorted([np.logical_and(filtered_sets[i], np.logical_not(covered_elements)) for i in range(len(filtered_sets))], key=lambda x: sum(x))
      best_set = filtered_sets.pop()

      missing_elements -= sum(best_set)
      steps += 1
      covered_elemts = best_set

    return steps if missing_elements <= 0 else PROBLEM_SIZE

def gready_cost(state):
    return distance(state) + len(state[0])

def A_cost1(state):
    return h1(state) + len(state[0])

def A_cost2(state):
    return h2(state) + len(state[0])

def A_cost3(state):
    return h3(state) + len(state[0])

def A_cost4(state):
    return h4(state) + len(state[0])

def breath_cost(state):
    return len(state[0])

In [48]:
def search(initial_state, cost_function):
        print(cost_function.__name__)
        frontier = PriorityQueue()
        state = initial_state
        counter = 0
        if(not goal_check((state[1], state[0]))):
            print("\tNo solution found")
        else:
          start = time.time()
          while state[1] and not goal_check(state):
              counter += 1
              for action in state[1]:
                  if action not in state[0]:
                      new_state = (state[0] ^ {action}, state[1] ^ {action})
                      frontier.put((cost_function(new_state), new_state))
              _, state = frontier.get()
          end = time.time()

          print("\tSolution:", state[0], _)
          print("\tSteps: ", counter)
          print(f'\tTime for step: {((end - start)/counter):.2e}')


In [57]:
for cost_fun in [gready_cost, A_cost3, A_cost4]:
    search(initial_state, cost_fun)

gready_cost
	Solution: {3, 15, 17, 18, 22, 24, 29} 7
	Steps:  13
	Time for step: 1.63e-03
A_cost3
	Solution: {3, 40, 43, 14, 48, 17} 6
	Steps:  10389
	Time for step: 2.18e-02
A_cost4
	Solution: {3, 40, 43, 14, 48, 17} 6
	Steps:  10389
	Time for step: 5.38e-02
